<a href="https://colab.research.google.com/github/szilaard/AIT_project/blob/main/AitProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIT Deep Learning Project

Péter Czumbel

Szilárd Horváth


In [ ]:
import tensorflow as tf
import librosa
import pandas as pd
from glob import glob
import IPython
import IPython.display as ipd
import numpy as np

Downloading the GTZAN dataset from tensorflow datasets doesn't work, the URL times out.<br>
See: https://github.com/tensorflow/datasets/issues/4090 <br>
Using [this](https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification) version of the dataset from kaggle instead.



We read all the data from the directories with glob.

In [ ]:
audio_files = glob("Data/genres_original/*/*.wav")

Example audio:

In [ ]:
ipd.Audio(audio_files[0])

We read the raw data of the first audio sample and its sample frequency

In [ ]:
signal, sr = librosa.load(audio_files[0])
print("Y is a numpy array:", signal)
print("Shape of Y:", signal.shape)
print("Sample Rate:", sr)

Fourier transformation

In [ ]:
D = librosa.stft(signal)

S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

print("New shape of transformed data", S_db.shape)